In [17]:
import os
from dotenv import load_dotenv
import sqlite3
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich import box

In [18]:
# Load environment variables
load_dotenv()

console = Console()

In [19]:
DB_FILE = os.getenv("DB_FILE", "game_database.db")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [32]:
def get_connection():
    conn = sqlite3.connect(DB_FILE)
    # If using SQLCipher, uncomment the following lines after installing pysqlcipher3
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA key = '{DB_PASSWORD}';")
    cursor.close()
    return conn

In [33]:
def execute_query(query: str):
    try:
        conn = get_connection()
        cursor = conn.cursor()

        console.print("[bold cyan]📡 Executing Query...[/bold cyan]")
        cursor.execute(query)

        if query.strip().lower().startswith("select"):
            rows = cursor.fetchall()
            if rows:
                columns = [description[0] for description in cursor.description]

                table = Table(title="Query Results", box=box.ROUNDED, show_lines=True)
                for col in columns:
                    table.add_column(col, style="bold magenta")

                for row in rows:
                    table.add_row(*[str(cell) for cell in row])

                console.print(table)
                console.print(f"✅ [green]{len(rows)} rows fetched successfully.[/green]")
            else:
                console.print(Panel("[yellow]⚠️ Query returned no rows.[/yellow]", title="No Data", style="yellow"))
        else:
            conn.commit()
            console.print(Panel(f"✅ [green]Query executed successfully.[/green]", title="Success", style="green"))

    except Exception as e:
        console.print(Panel(f"❗ [red]Error executing query:[/red] {str(e)}", title="Error", style="red"))

    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [34]:
def list_tables():
    """List all tables with row counts."""
    query = """
    SELECT name FROM sqlite_master 
    WHERE type='table' AND name NOT LIKE 'sqlite_%'
    ORDER BY name;
    """
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(query)
        tables = cursor.fetchall()

        if not tables:
            console.print(Panel("[yellow]⚠️ No tables found in the database.[/yellow]", title="No Tables", style="yellow"))
            return

        table = Table(title="Tables in Database", box=box.ROUNDED, show_lines=True)
        table.add_column("Table Name", style="bold green")
        table.add_column("Number of Records", style="bold cyan", justify="right")

        for (table_name,) in tables:
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            table.add_row(f"📋 {table_name}", str(count))

        console.print(table)
    except Exception as e:
        console.print(Panel(f"❗ [red]Error listing tables:[/red] {str(e)}", title="Error", style="red"))
    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [35]:
def table_info(table_name: str):
    """Show info about table: columns, row count, and sample rows (max 10)."""
    try:
        conn = get_connection()
        cursor = conn.cursor()

        # Check if table exists
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if not cursor.fetchone():
            console.print(Panel(f"⚠️ Table '{table_name}' does not exist.", style="yellow"))
            return

        # Get columns info
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns_info = cursor.fetchall()
        columns = [col[1] for col in columns_info]

        # Get row count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]

        # Print columns and row count
        info_panel = Panel.fit(
            f"📋 [bold green]Table:[/bold green] {table_name}\n"
            f"🔢 [bold cyan]Columns:[/bold cyan] {', '.join(columns)}\n"
            f"📊 [bold magenta]Total Rows:[/bold magenta] {row_count}",
            title="Table Info",
            style="blue",
        )
        console.print(info_panel)

        # Fetch and show up to 10 rows
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 10")
        rows = cursor.fetchall()

        if rows:
            table = Table(title=f"Sample Data from {table_name}", box=box.ROUNDED, show_lines=True)
            for col in columns:
                table.add_column(col, style="bold magenta")
            for row in rows:
                table.add_row(*[str(cell) for cell in row])
            console.print(table)
        else:
            console.print(Panel("[yellow]⚠️ No data found in the table.[/yellow]", title="Empty Table", style="yellow"))

    except Exception as e:
        console.print(Panel(f"❗ [red]Error fetching table info:[/red] {str(e)}", title="Error", style="red"))
    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [40]:
list_tables()

╭─────────────────────────────────────────────────── No Tables ───────────────────────────────────────────────────╮
│ ⚠️ No tables found in the database.                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [41]:
def drop_tables_interactive():
    conn = get_connection()
    cursor = conn.cursor()
    
    # Get list of user tables in SQLite
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
    tables = [row[0] for row in cursor.fetchall()]
    
    if not tables:
        print("No user tables found to drop.")
        cursor.close()
        conn.close()
        return
    
    print("\nExisting tables:")
    for idx, table in enumerate(tables, 1):
        print(f"{idx}. {table}")
    
    print("\nOptions:")
    print("  Enter the table number to drop a specific table")
    print("  Enter 'all' to drop ALL tables")
    print("  Enter 'exit' to cancel and exit")
    
    while True:
        choice = input("\nYour choice: ").strip().lower()
        
        if choice == 'exit':
            print("Exit without dropping tables.")
            break
        
        elif choice == 'all':
            for table in tables:
                cursor.execute(f"DROP TABLE IF EXISTS {table};")
                print(f"✅ Dropped table '{table}'")
            conn.commit()
            break
        
        elif choice.isdigit():
            idx = int(choice)
            if 1 <= idx <= len(tables):
                table = tables[idx - 1]
                cursor.execute(f"DROP TABLE IF EXISTS {table};")
                conn.commit()
                print(f"✅ Dropped table '{table}'")
                # Update tables list after drop
                tables.pop(idx - 1)
                if not tables:
                    print("No more tables left.")
                    break
                else:
                    print("\nRemaining tables:")
                    for idx2, t in enumerate(tables, 1):
                        print(f"{idx2}. {t}")
            else:
                print("Invalid table number, try again.")
        else:
            print("Invalid input, try again.")
    
    cursor.close()
    conn.close()

In [42]:
drop_tables_interactive()

No user tables found to drop.


## 1.1 🧑‍💼 1. Players Table

In [51]:
query_players = """
CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL UNIQUE,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    avatar_id INTEGER,                                -- Foreign key to avatars
    confirmation_token TEXT,
    is_confirmed INTEGER DEFAULT 0,
    reset_token TEXT,
    role TEXT CHECK (role IN ('admin', 'player')) DEFAULT 'player',
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,
    last_login_at TEXT,
    FOREIGN KEY (avatar_id) REFERENCES avatars(id) ON DELETE SET NULL
);
"""
execute_query(query_players)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 2.2 🏆 2. Leagues Table

In [66]:
query_leagues = """
CREATE TABLE IF NOT EXISTS leagues (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE NOT NULL,
    country TEXT,
    logo_path TEXT,                          -- Path to local league logo
    can_be_draw INTEGER DEFAULT 1,           -- 1 = draw allowed
    two_legs INTEGER DEFAULT 0,              -- 1 = two-legged ties
    must_have_winner INTEGER DEFAULT 0,      -- 1 = no draw allowed (e.g., finals)
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""
execute_query(query_leagues)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 3.3 📅 3. Rounds Table

In [53]:
query_rounds = """
CREATE TABLE IF NOT EXISTS rounds (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    start_date TEXT NOT NULL,
    end_date TEXT NOT NULL,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);
"""
execute_query(query_rounds)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 4.4 🏟️ 4. Teams Table

In [ ]:
query_teams = """
CREATE TABLE IF NOT EXISTS teams (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    logo_path TEXT,
    league_id INTEGER NOT NULL,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE
);
"""
execute_query(query_teams)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## team_league join table

In [67]:
execute_query("""
CREATE TABLE IF NOT EXISTS team_league (
    team_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    PRIMARY KEY (team_id, league_id),
    FOREIGN KEY (team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE
);
""")

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 5.5 ⚽ 5. Matches Table

In [ ]:
query_matches = """
CREATE TABLE IF NOT EXISTS matches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    round_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    home_team_id INTEGER NOT NULL,
    away_team_id INTEGER NOT NULL,
    match_datetime TEXT NOT NULL,
    status TEXT CHECK (status IN ('upcoming', 'live', 'finished', 'cancelled')) DEFAULT 'upcoming',
    FOREIGN KEY (round_id) REFERENCES rounds(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE,
    FOREIGN KEY (home_team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (away_team_id) REFERENCES teams(id) ON DELETE CASCADE
);
"""
execute_query(query_matches)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 6.6 🔁 6. Legs Table

In [56]:
query_legs = """
CREATE TABLE IF NOT EXISTS legs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    match_id INTEGER NOT NULL,
    leg_number INTEGER NOT NULL,
    leg_date TEXT NOT NULL,
    home_score INTEGER,
    away_score INTEGER,
    can_draw BOOLEAN DEFAULT 1,
    winner_team_id INTEGER,
    notes TEXT,
    FOREIGN KEY (match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (winner_team_id) REFERENCES teams(id) ON DELETE SET NULL,
    UNIQUE (match_id, leg_number)
);
"""
execute_query(query_legs)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 7.7 🖼️ 7. Avatars Table

In [57]:
query_avatars = """
CREATE TABLE IF NOT EXISTS avatars (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    file_path TEXT NOT NULL,                          -- Local path to the image
    uploaded_by_player_id INTEGER,                    -- Who uploaded the avatar
    uploaded_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (uploaded_by_player_id) REFERENCES players(id) ON DELETE SET NULL
);
"""
execute_query(query_avatars)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 8.8 predictions Table

In [63]:
query_predictions = """
CREATE TABLE IF NOT EXISTS predictions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER NOT NULL,
    match_id INTEGER NOT NULL,
    
    predicted_home_score INTEGER NOT NULL,
    predicted_away_score INTEGER NOT NULL,
    predicted_penalty_winner TEXT,                -- Team name or NULL
    
    score INTEGER DEFAULT 0,                      -- Computed after match ends

    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    
    FOREIGN KEY (player_id) REFERENCES players(id) ON DELETE CASCADE,
    FOREIGN KEY (match_id) REFERENCES matches(id) ON DELETE CASCADE,
    
    UNIQUE (player_id, match_id)                  -- Player can only predict once per match
);
"""
execute_query(query_predictions)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 9.9 achievements Table

In [64]:
query_achievements = """
CREATE TABLE IF NOT EXISTS achievements (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER NOT NULL,
    
    total_leagues_won INTEGER DEFAULT 0,
    total_cups_won INTEGER DEFAULT 0,

    updated_at TEXT DEFAULT CURRENT_TIMESTAMP,

    FOREIGN KEY (player_id) REFERENCES players(id) ON DELETE CASCADE,
    UNIQUE (player_id)
);
"""
execute_query(query_achievements)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [65]:
list_tables()

          Tables in Database           
╭─────────────────┬───────────────────╮
│ Table Name      │ Number of Records │
├─────────────────┼───────────────────┤
│ 📋 achievements │                 0 │
├─────────────────┼───────────────────┤
│ 📋 avatars      │                 0 │
├─────────────────┼───────────────────┤
│ 📋 leagues      │                 0 │
├─────────────────┼───────────────────┤
│ 📋 legs         │                 0 │
├─────────────────┼───────────────────┤
│ 📋 matches      │                 0 │
├─────────────────┼───────────────────┤
│ 📋 players      │                 0 │
├─────────────────┼───────────────────┤
│ 📋 predictions  │                 0 │
├─────────────────┼───────────────────┤
│ 📋 rounds       │                 0 │
├─────────────────┼───────────────────┤
│ 📋 teams        │                 0 │
╰─────────────────┴───────────────────╯

🔌 Connection closed.

In [62]:
table_info("leagues")

╭──────────────────────────────────────────────── Table Info ────────────────────────────────────────────────╮
│ 📋 Table: leagues                                                                                          │
│ 🔢 Columns: id, name, country, default_leg_count, default_can_draw, default_tie_break_required, created_at │
│ 📊 Total Rows: 0                                                                                           │
╰────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.